In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization

In [2]:
import os
print(os.listdir("data"))

['.DS_Store', 'large1.csv', 'large2.csv', 'small1.csv', 'small2.csv']


In [3]:
def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))




# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(input_quiz, N=9):
    m = np.reshape([int(c) for c in input_quiz], (N,N))
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))
    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr()     
    return CLUE

## Small1

In [16]:
from cvxopt import solvers, matrix
import time
solvers.options['show_progress'] = False

data = pd.read_csv("data/small1.csv") 

corr_cnt = 0
start = time.time()
for i in range(len(data)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    def solver(quiz, solu):
        A0 = fixed_constraints()
        A1 = clue_constraint(quiz)

        # Formulate the matrix A and vector B (B is all ones).
        A = scs.vstack((A0,A1))
        A = A.toarray()
        B = np.ones(A.shape[0])

        # Because rank defficiency. We need to extract effective rank.
        u, s, vh = np.linalg.svd(A, full_matrices=False)
        K = np.sum(s > 1e-12)
        S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
        A = S_@vh
        B = u.T@B
        B = B[:K]

        c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
        G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                             [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
        h = matrix(np.zeros(A.shape[1]*2))
        H = matrix(np.block([A, -A]))
        b = matrix(B)
        
        sol = solvers.lp(c,G,h,H,b)
        X = np.array(sol['x']).T[0]
        x = X[:A.shape[1]] - X[A.shape[1]:]

        # map to board
        z = np.reshape(x, (81, 9))
        
        
        return z
    
    # Implementing recursion method to delete repeated variable 
    
    z = solver(quiz, solu)
    D = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9))
    if np.linalg.norm(D, np.inf) > 0: # Checking if solution is correct
        New = np.array([np.argmax(d)+1 for d in z]) # sudoku solution in 1d array form
        D = D.reshape(-1) # array with zero or nonzero
        for j in range(len(D)):
            if D[j] != 0: 
                New[j] = 0 # replace repeated variable with zero
        new_z = solver(New, solu) # solve new puzzle using LP 
        if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in new_z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9)), np.inf) > 0:
            pass
        else:
            #print("CORRECT",i)
            corr_cnt += 1
        
    else:
        #print("CORRECT",i)
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )
end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.90 secs. Success rate: 20 / 20 
Aver Time:   0.89 secs. Success rate: 24 / 24 


## Small2

In [13]:
data = pd.read_csv("data/small2.csv") 

corr_cnt = 0
start = time.time()
for i in range(len(data)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    def solver(quiz, solu):
        A0 = fixed_constraints()
        A1 = clue_constraint(quiz)

        # Formulate the matrix A and vector B (B is all ones).
        A = scs.vstack((A0,A1))
        A = A.toarray()
        B = np.ones(A.shape[0])

        # Because rank defficiency. We need to extract effective rank.
        u, s, vh = np.linalg.svd(A, full_matrices=False)
        K = np.sum(s > 1e-12)
        S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
        A = S_@vh
        B = u.T@B
        B = B[:K]

        c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
        G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                             [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
        h = matrix(np.zeros(A.shape[1]*2))
        H = matrix(np.block([A, -A]))
        b = matrix(B)
        
        sol = solvers.lp(c,G,h,H,b)
        X = np.array(sol['x']).T[0]
        x = X[:A.shape[1]] - X[A.shape[1]:]

        # map to board
        z = np.reshape(x, (81, 9))

        return z
    
    # Implementing recursion method to delete repeated variable 
    
    z = solver(quiz, solu)
    D = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9))
    if np.linalg.norm(D, np.inf) > 0:
        New = np.array([np.argmax(d)+1 for d in z])
        D = D.reshape(-1)
        for j in range(len(D)):
            if D[j] != 0:
                New[j] = 0
        new_z = solver(New, solu)
        if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in new_z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9)), np.inf) > 0:
            pass
        else:
            #print("CORRECT",i)
            corr_cnt += 1
        
    else:
        #print("CORRECT",i)
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )
end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   1.18 secs. Success rate: 20 / 20 
Aver Time:   1.18 secs. Success rate: 40 / 40 
Aver Time:   1.17 secs. Success rate: 60 / 60 
Aver Time:   1.17 secs. Success rate: 80 / 80 
Aver Time:   1.19 secs. Success rate: 100 / 100 
Aver Time:   1.19 secs. Success rate: 120 / 120 
Aver Time:   1.22 secs. Success rate: 140 / 140 
Aver Time:   1.23 secs. Success rate: 160 / 160 
Aver Time:   1.24 secs. Success rate: 180 / 180 
Aver Time:   1.24 secs. Success rate: 200 / 200 
Aver Time:   1.25 secs. Success rate: 220 / 220 
Aver Time:   1.25 secs. Success rate: 240 / 240 
Aver Time:   1.25 secs. Success rate: 260 / 260 
Aver Time:   1.26 secs. Success rate: 280 / 280 
Aver Time:   1.26 secs. Success rate: 300 / 300 
Aver Time:   1.26 secs. Success rate: 320 / 320 
Aver Time:   1.27 secs. Success rate: 340 / 340 
Aver Time:   1.26 secs. Success rate: 360 / 360 
Aver Time:   1.27 secs. Success rate: 380 / 380 
Aver Time:   1.28 secs. Success rate: 400 / 400 
Aver Time:   1.28 secs. Succ

## Large1

In [15]:
data = pd.read_csv("data/large1.csv") 

corr_cnt = 0
start = time.time()

random_seed = 42
np.random.seed(random_seed)

if len(data) > 1000:
    samples = np.random.choice(len(data), 1000)
else:
    samples = range(len(data))
    
for i in range(len(samples)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    def solver(quiz, solu):
        A0 = fixed_constraints()
        A1 = clue_constraint(quiz)

        # Formulate the matrix A and vector B (B is all ones).
        A = scs.vstack((A0,A1))
        A = A.toarray()
        B = np.ones(A.shape[0])

        # Because rank defficiency. We need to extract effective rank.
        u, s, vh = np.linalg.svd(A, full_matrices=False)
        K = np.sum(s > 1e-12)
        S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
        A = S_@vh
        B = u.T@B
        B = B[:K]

        c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
        G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                             [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
        h = matrix(np.zeros(A.shape[1]*2))
        H = matrix(np.block([A, -A]))
        b = matrix(B)
        
        sol = solvers.lp(c,G,h,H,b)
        X = np.array(sol['x']).T[0]
        x = X[:A.shape[1]] - X[A.shape[1]:]

        # map to board
        z = np.reshape(x, (81, 9))

        return z
    
    # Implementing recursion method to delete repeated variable 

    z = solver(quiz, solu)
    D = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9))
    if np.linalg.norm(D, np.inf) > 0:
        New = np.array([np.argmax(d)+1 for d in z])
        D = D.reshape(-1)
        for j in range(len(D)):
            if D[j] != 0:
                New[j] = 0
        new_z = solver(New, solu)
        if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in new_z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9)), np.inf) > 0:
            pass
        else:
            #print("CORRECT",i)
            corr_cnt += 1
        
    else:
        #print("CORRECT",i)
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )
end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   1.05 secs. Success rate: 20 / 20 
Aver Time:   1.01 secs. Success rate: 40 / 40 
Aver Time:   1.02 secs. Success rate: 60 / 60 
Aver Time:   1.02 secs. Success rate: 80 / 80 
Aver Time:   1.01 secs. Success rate: 100 / 100 
Aver Time:   1.00 secs. Success rate: 120 / 120 
Aver Time:   0.99 secs. Success rate: 140 / 140 
Aver Time:   0.99 secs. Success rate: 160 / 160 
Aver Time:   0.99 secs. Success rate: 180 / 180 
Aver Time:   0.98 secs. Success rate: 200 / 200 
Aver Time:   0.98 secs. Success rate: 220 / 220 
Aver Time:   0.98 secs. Success rate: 240 / 240 
Aver Time:   0.97 secs. Success rate: 260 / 260 
Aver Time:   0.98 secs. Success rate: 280 / 280 
Aver Time:   0.98 secs. Success rate: 300 / 300 
Aver Time:   0.97 secs. Success rate: 320 / 320 
Aver Time:   0.98 secs. Success rate: 340 / 340 
Aver Time:   0.98 secs. Success rate: 360 / 360 
Aver Time:   0.98 secs. Success rate: 380 / 380 
Aver Time:   0.98 secs. Success rate: 400 / 400 
Aver Time:   0.98 secs. Succ

## Large2

In [14]:
data = pd.read_csv("data/large2.csv") 

corr_cnt = 0
start = time.time()

random_seed = 42
np.random.seed(random_seed)

if len(data) > 1000:
    samples = np.random.choice(len(data), 1000)
else:
    samples = range(len(data))
    
for i in range(len(samples)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    def solver(quiz, solu):
        A0 = fixed_constraints()
        A1 = clue_constraint(quiz)

        # Formulate the matrix A and vector B (B is all ones).
        A = scs.vstack((A0,A1))
        A = A.toarray()
        B = np.ones(A.shape[0])

        # Because rank defficiency. We need to extract effective rank.
        u, s, vh = np.linalg.svd(A, full_matrices=False)
        K = np.sum(s > 1e-12)
        S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
        A = S_@vh
        B = u.T@B
        B = B[:K]

        c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
        G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                             [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
        h = matrix(np.zeros(A.shape[1]*2))
        H = matrix(np.block([A, -A]))
        b = matrix(B)
        
        sol = solvers.lp(c,G,h,H,b)
        X = np.array(sol['x']).T[0]
        x = X[:A.shape[1]] - X[A.shape[1]:]

        # map to board
        z = np.reshape(x, (81, 9))

        return z
    
    # Implementing recursion method to delete repeated variable 
    
    z = solver(quiz, solu)
    D = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9))
    if np.linalg.norm(D, np.inf) > 0:
        New = np.array([np.argmax(d)+1 for d in z])
        D = D.reshape(-1)
        for j in range(len(D)):
            if D[j] != 0:
                New[j] = 0
        new_z = solver(New, solu)
        if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in new_z]), (9,9) ) \
        - np.reshape([int(c) for c in solu], (9,9)), np.inf) > 0:
            pass
        else:
            #print("CORRECT",i)
            corr_cnt += 1
        
    else:
        #print("CORRECT",i)
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )
end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.81 secs. Success rate: 20 / 20 
Aver Time:   0.78 secs. Success rate: 40 / 40 
Aver Time:   0.78 secs. Success rate: 60 / 60 
Aver Time:   0.78 secs. Success rate: 80 / 80 
Aver Time:   0.78 secs. Success rate: 100 / 100 
Aver Time:   0.78 secs. Success rate: 120 / 120 
Aver Time:   0.81 secs. Success rate: 140 / 140 
Aver Time:   0.81 secs. Success rate: 160 / 160 
Aver Time:   0.81 secs. Success rate: 180 / 180 
Aver Time:   0.82 secs. Success rate: 200 / 200 
Aver Time:   0.83 secs. Success rate: 220 / 220 
Aver Time:   0.83 secs. Success rate: 240 / 240 
Aver Time:   0.82 secs. Success rate: 260 / 260 
Aver Time:   0.82 secs. Success rate: 280 / 280 
Aver Time:   0.82 secs. Success rate: 300 / 300 
Aver Time:   0.82 secs. Success rate: 320 / 320 
Aver Time:   0.82 secs. Success rate: 340 / 340 
Aver Time:   0.82 secs. Success rate: 360 / 360 
Aver Time:   0.82 secs. Success rate: 380 / 380 
Aver Time:   0.81 secs. Success rate: 400 / 400 
Aver Time:   0.81 secs. Succ